In [2]:
# Indlæser data brugt i øvelserne

import pandas as pd

reddit_df = pd.read_csv("https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/reddit_rdenmark-comments_01032021-08032021_long.csv")

## ØVELSE 1: Sentiment Analysis

Foretag sentiment analysis enten på eget data eller på r/Denmark data: [reddit_rdenmark-comments_01032021-08032021_long.csv](https://raw.githubusercontent.com/CALDISS-AAU/course_ddf/master/datasets/reddit_rdenmark-comments_01032021-08032021_long.csv)

(DaCy/senda for dansk, TextBlob for engelsk)

**Bemærk:** Sentiment analysis med især DaCy kan tage lang tid på større datasæt. I kan med fordel arbejde med et subset i denne øvelse, for at teste funktionerne af.

1. Anvend sentiment analysis på enkelte tekststykker
2. Anvend sentiment analysis på et subset (enten med egne betingelser eller med [`pd.sample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html))
3. Foretag opsummering af subsettet, der giver indikation af, hvorvidt tonen i materialet er overvejende positiv eller negativ

### (En) løsning

In [3]:
# Indlæser DaCy
import dacy
from dacy.sentiment import add_senda

# Indlæser sprogmodel
nlp = dacy.load("da_dacy_medium_tft-0.0.0")

# Tilføjer senda sentiment anlaysis moel til pipeline
add_senda(nlp, force_extension = True)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/tljh/user/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'da_dacy_medium_tft' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/tljh/user/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [4]:
# Udvælger tilfældig kommentar

a_comment = reddit_df.loc[376, 'comment_body']
a_comment

'Du synes ikke det er forståeligt at folk er chokerede over en prisstigning fra 309kr til 419kr på sportspakken? Det er en stigning på 35%. Det er fuldstændig vanvittigt for en tjeneste der er rigeligt dyr i forvejen.'

In [5]:
# Bruger sentiment analysis på kommentar:

doc = nlp(a_comment)

print(doc._.polarity)

negative


In [6]:
# Danner sample med 15 kommentarer med pd.sample

reddit_sample = reddit_df.sample(n = 15)

In [7]:
# Danner wrapper funktion

def simple_sentiment(text):
    doc = nlp(text)
    polarity = doc._.polarity
    
    return(polarity)

In [8]:
# Bruger wrapper på sample

reddit_sample['sentiment'] = reddit_sample['comment_body'].apply(simple_sentiment)

In [9]:
# Opsummering/optælling

reddit_sample['sentiment'].value_counts()

negative    10
neutral      3
positive     2
Name: sentiment, dtype: int64

## ØVELSE 2: Simpel teksthåndtering

I øvelserne i dag skal i arbejde med et datasæt bestående af kommentarer fra reddit. Alle kommentarer er taget fra posts på r/denmark (reddit.com/r/denmark) fra 1/3-8/3 2021.

1. Indlæs data som en pandas data frame
    - Link til data: https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/reddit_rdenmark-comments_01032021-08032021_long.csv
2. Dan et subset bestående af alle kommentarer, der nævner "menneskerettigheder" (kommentarteksten er i kolonnen `comment_body`). Hvor mange kommentarer er der?

**Bonus**
- Kan du udregne gennemsnitsscore for de kommentarer, der nævner menneskerettigheder? (score fremgår af kolonnen `comment_score`)

### (En) løsning

In [10]:
# Subset med alle kommentarerne "menneskerettigheder"

reddit_subset = reddit_df.loc[reddit_df['comment_body'].str.contains("menneskerettigheder"), :]

# Alternativ - både med lille og stort for bogstav
reddit_subset = reddit_df.loc[reddit_df['comment_body'].str.contains("menneskerettigheder") | reddit_df['comment_body'].str.contains("Menneskerettigheder"), :]

# Alternativ - omdanner til lower-case først
reddit_df['comment_body_l'] = reddit_df['comment_body'].str.lower()
reddit_subset = reddit_df.loc[reddit_df['comment_body'].str.contains("menneskerettigheder"), :]

In [11]:
# Antal kommentarer (ud fra hvor mange rækker, der er)

reddit_subset.shape

(11, 53)

In [12]:
# Bonus: Gennemsnitsscore for kommentarerne

reddit_subset['comment_score'].mean()

35.45454545454545

## ØVELSE 3: Brug af sprogmodel

1. Udvælg en enkelt kommentar fra reddit datasættet. Kommentarteksten findes i kolonnen "comment_body" (fx `comment = reddit_df.loc[200, 'comment_body']`)

2. Analysér kommentaren med spaCy sprogmodellen (omdan kommentaren til et `doc` objekt). Husk at installér og indlæs sprogmodellen først:

```python
import spacy

!python -m spacy download da_core_news_sm
nlp = spacy.load("da_core_news_sm")
```

3. Er der named entities i kommentaren? (`doc.ents`) I så fald hvilke?

**Bonus**

- Lav en liste, der kun indeholder ord fra kommentaren med ordklassen "NOUN"

### (En) løsning

In [13]:
# Indlæser spacy og dansk sprogmodel
import spacy
nlp = spacy.load("da_core_news_sm")

In [14]:
# Udvælg kommentar fra reddit sæt
comment = reddit_df.loc[200, 'comment_body']
comment

'Jeg tænker EB gør det, fordi at der er et hårdt rygte på, at Lars Løkke er ved at starte eget parti op.'

In [15]:
# Brug sprogmodel på kommentar
doc = nlp(comment)

In [16]:
# Er der named entities? (Ja)

doc.ents

(Lars Løkke,)

In [17]:
# Bonus: Liste kun med navneord i kommentaren

nouns = [] 

for word in doc:
    if word.pos_ == "NOUN":
        nouns.append(word.text)
        
nouns

['parti']

## ØVELSE 4: Tidy text data (reddit data)

Du skal nu tokenize alle *kommentarerne* i reddit datasættet (kolonnen `comment_body`)

1. Brug `.apply()` til at anvende tokenizer funktion på hele reddit datasættet til at lave en tokens kolonne (det kan være en god ide at teste funktionen med en enkelt kommentar først)
2. Brug `.explode()` til at konvertere data til et tidy format
3. Brug `.value_counts()` til at optælle tokens

**Bonus**

- Undersøg, hvor mange gange coronavirus er nævnt (tænkt gerne synonymer med!)

### (En) løsning

In [18]:
# Simpel tokenizer funktion (kopieret fra notebook 07)
from spacy.lang.da import Danish
nlp = Danish() # Indlæser "tom" sprogmodel
tokenizer = nlp.tokenizer # henter tokenizer

def tokenizer_simple(text):
    stop_words = list(nlp.Defaults.stop_words)
    
    doc = tokenizer(text)
    
    tokens = []
    
    for word in doc:
        if len(word.text) < 3:
            continue
        if word.text not in stop_words:
            tokens.append(word.text)
    
    return(tokens)

In [19]:
# Tokenize alle kommentarer - dan variabel med tokens
reddit_df['tokens'] = reddit_df['comment_body'].apply(tokenizer_simple)

In [20]:
# Omdan til tidy format (en række per ord)

reddit_df = reddit_df.explode('tokens')

In [21]:
# Optæl tokens
reddit_df['tokens'].value_counts()

Det            955
Jeg            716
the            414
bare           345
godt           251
              ... 
Seier            1
uvidst           1
Retorikken       1
strømninger      1
pubeshår         1
Name: tokens, Length: 15571, dtype: int64

In [23]:
# Bonus: hvor mange gange er coronavirus eller lignende nævnt? (65 gange baseret på antal rækker efter filtrering på nøgleord)

coronawords = ['corona', 'coronavirus', 'covid', 'covid-19', 'covid19']

reddit_df.loc[reddit_df['tokens'].str.lower().isin(coronawords), :].shape

(65, 54)